In [5]:
# 数据清洗

In [1]:
import glob
import pandas as pd
from pathlib import Path
import re

In [2]:
from PIL import Image

In [3]:
model_label_map = {
    0: 10,  # 鞋子
    1: 8,  # 包包
    2: 4,  # 上装
    3: 5,  # 裤子
    4: 6,  # 裙子
    5: 7,  # 连体装
}

In [4]:
model_label_map = {
    0: 'shoe',
    1: 'bag',
    2: 'upperbody',
    3: 'lowerbody',  # 裤子
    4: 'skirt',  # 裙子
    5: 'wholebody',  # 连体装
}

### LOAD DATA

In [7]:
clred_df = pd.read_csv('/nas/chenyi/datasets_cls/gallery_detect/gallery_oss/dadetv4/info/clred_labels_oss.csv')

In [9]:
# clred_df.to_csv('/nas/chenyi/datasets_cls/gallery_detect/gallery_oss/dadetv4/info/clred_labels_oss.csv', index=False)
# clred_final_df.to_csv('./clred_temp_final.csv', index=False)

In [11]:
clred_df.head(1).values

array([[0, 0.373611, 0.942361, 0.0833333, 0.115278, 0.880866, 0,
        '/nas/chenyi/datasets_cls/gallery_detect/gallery_oss/dadetv4/gall_dadetv4_all/labels/69313640-af52-465e-88c8-c0a0099ca5c7-699544.txt',
        '69313640-af52-465e-88c8-c0a0099ca5c7-699544',
        '/nas/oss/gallery/69/31/69313640-af52-465e-88c8-c0a0099ca5c7-699544.jpg',
        933, 933, 'shoe',
        '/nas/chenyi/datasets_cls/gallery_detect/gallery_oss/dadetv4/gall_dadetv4_all/crops/shoe/69313640-af52-465e-88c8-c0a0099ca5c7-699544_0.jpg',
        '/nas/chenyi/datasets_cls/gallery_detect/gallery_oss/dadetv4/gall_dadetv4_all/crops_oss/69/31/69313640-af52-465e-88c8-c0a0099ca5c7-699544_0.jpg']],
      dtype=object)

In [8]:
# %%time
# crops_less_imgs = glob.glob(str('/nas/chenyi/datasets_cls/gallery_detect/gallery_oss/dadetv4/gall_dadetv4_all/crops/*/*'))
# len(crops_less_imgs)

#### crops target path.

In [12]:
!ls /nas/chenyi/datasets_cls/gallery_detect/gallery_oss/dadetv4/gall_dadetv4_all/crops/

bag  lowerbody	shoe  skirt  upperbody	wholebody


In [24]:
clred_final_df['label_eng'] = clred_final_df['label_pred'].map(model_label_map)

In [29]:
crops_root = '/nas/chenyi/datasets_cls/gallery_detect/gallery_oss/dadetv4/gall_dadetv4_all/crops'
clred_final_df['crops_sp'] = clred_final_df.apply(lambda row:
                                                '{0}/{1}/{2}_{3}.jpg'.format(crops_root, row['label_eng'], row['stem'], row['id_crop']),
                                                axis=1)

In [37]:
# p = clred_final_df['crops_sp'][110]
# !ls $p

In [39]:
crops_troot = '/nas/chenyi/datasets_cls/gallery_detect/gallery_oss/dadetv4/gall_dadetv4_all/crops_oss'
clred_final_df['crops_tp'] = clred_final_df.apply(lambda x:
                                                '{0}/{1}/{2}/{3}'.format(crops_troot, x['stem'][:2], x['stem'][2:4], Path(x['crops_sp']).name),
                                                axis=1)

### 图片清洗

In [3]:
clred_df = label_cross_df

#### 图片太小

In [4]:
len(label_cross_df[(label_cross_df['width']<300) | (label_cross_df['heigth']<300)])

8258

In [5]:
clred_df = clred_df[(clred_df['width']>300) | (clred_df['heigth']>300)]

In [6]:
thres = 500
len( clred_df[(clred_df['width']<thres) & (clred_df['heigth']<thres)])

46714

In [7]:
thres = 500
clred_df =  clred_df[(clred_df['width']>thres) & (clred_df['heigth']>thres)]

In [8]:
len(clred_df)

6302384

In [23]:
label_cross_df.value_counts('label_pred')

label_pred
0    2786713
2    1388800
3     812387
1     786189
5     489965
4     335122
dtype: int64

#### 去除置信度太小的

In [14]:
clred_df['conf_show'] = clred_df['conf'].apply(lambda x: round(x*10))

In [15]:
clred_df.value_counts('conf_show').sort_index()

conf_show
4      178808
5      461430
6      604511
7      780950
8     1221436
9     2559466
10     495783
dtype: int64

In [17]:
178808/len(clred_df)

0.028371486091612316

In [37]:
thres = 0.5
clred_df =  clred_df[(clred_df['conf']>=thres)]
len(label_cross_df), len(clred_df)

(6599176, 5891707)

#### 去掉图片目标太多的

In [38]:
len(label_cross_df[label_cross_df['id_crop']>7])

2125

In [39]:
thres = 7
bbox_much_df = clred_df[(clred_df['id_crop']>=thres)]
img_much_df = bbox_much_df.drop_duplicates('stem')

In [40]:
clred_df =  clred_df[(~clred_df['stem'].isin(img_much_df['stem']))]
len(label_cross_df), len(clred_df)

(6599176, 5891707)

### 处理bbox

In [136]:
# clred_df = pd.read_csv('clred_temp.csv')
clred_df.to_csv('clred_temp.csv', index=False)

In [83]:
# clred_df['bx_h'] = clred_df['heigth'].values*clred_df['bx4'].values
# clred_df['bx_w'] = clred_df['width'].values*clred_df['bx3'].values

In [84]:
# clred_df['h/w'] = clred_df['bx_h'].values/clred_df['bx_w'].values

#### 目标太小

In [86]:
thres = 20
clred_df = clred_df[(clred_df['bx_w']>thres) & (clred_df['bx_h']>thres)]

In [87]:
len(label_cross_df), len(clred_df)

(6599176, 5889317)

#### bbox h/w

In [97]:
thres = 4
temp = clred_df[(clred_df['h/w']>thres) | (clred_df['h/w']<1/thres)]
len(temp)

84820

In [98]:
temp.value_counts('label_pred')

label_pred
1    39461
0    23382
5     9366
3     9309
2     2585
4      717
dtype: int64

In [99]:
temp.value_counts('conf_show')

conf_show
9     31485
8     20557
7     13956
6     10967
5      4988
10     2867
dtype: int64

In [101]:
clred_df = clred_df[~clred_df['stem'].isin(temp['stem'])]
len(label_cross_df), len(clred_df), len(clred_df)/len(label_cross_df),

(6599176, 5450982, 0.8260094896696194)

In [114]:
clred_df.describe()

,label_pred,bx1,bx2,bx3,bx4,conf,id_crop,width,heigth,conf_show,bx_h,bx_w,h/w
count,5.323235e+06,5.323235e+06,5.323235e+06,5.323235e+06,5.323235e+06,5.323235e+06,5.323235e+06,5.323235e+06,5.323235e+06,5.323235e+06,5.323235e+06,5.323235e+06,5.323235e+06
mean,1.552688e+00,4.968524e-01,6.865291e-01,2.140302e-01,2.379825e-01,8.176063e-01,1.716938e+00,8.812612e+02,1.163139e+03,8.161885e+00,2.767135e+02,1.880899e+02,1.550196e+00
std,1.597507e+00,1.093304e-01,2.077841e-01,1.267241e-01,1.570488e-01,1.249757e-01,1.272721e+00,2.883422e+02,3.644001e+02,1.283396e+00,2.099181e+02,1.312383e+02,7.051437e-01
min,0.000000e+00,2.000000e-02,1.581720e-02,4.000000e-02,2.325580e-02,5.000000e-01,0.000000e+00,5.010000e+02,5.010000e+02,5.000000e+00,2.062748e+01,2.056804e+01,2.500000e-01
25%,0.000000e+00,4.411760e-01,5.062960e-01,1.086960e-01,1.034990e-01,7.372370e-01,1.000000e+00,6.900000e+02,9.200000e+02,7.000000e+00,1.146670e+02,9.072000e+01,1.018361e+00
50%,1.000000e+00,4.980450e-01,6.880170e-01,1.855070e-01,1.932370e-01,8.587160e-01,2.000000e+00,6.900000e+02,1.035000e+03,9.000000e+00,2.137585e+02,1.550002e+02,1.417156e+00
75%,3.000000e+00,5.528990e-01,8.985510e-01,2.929690e-01,3.472710e-01,9.187500e-01,3.000000e+00,1.252000e+03,1.441000e+03,9.000000e+00,3.809996e+02,2.465190e+02,1.968069e+00
max,5.000000e+00,9.797770e-01,9.800000e-01,1.000000e+00,1.000000e+00,9.866390e-01,6.000000e+00,8.256000e+03,8.192000e+03,1.000000e+01,5.192221e+03,3.150000e+03,4.000000e+00


#### 鞋子

In [41]:
# 都是模特图
# 鞋子都比较小,绝对值和相对值
# 都在下半图，且可能比较大

In [241]:
# clred_df = pd.read_csv('clred_temp.csv')
clred_df.to_csv('clred_final.csv', index=False)

In [170]:
cls = 5

##### 鞋子太小

In [171]:
thres = 50
temp = clred_df[(clred_df['bx_h']<thres) | (clred_df['bx_w']<thres) 
             & (clred_df['label_pred']==cls)]
len(temp)

59545

In [172]:
clred_df = clred_df[~clred_df['stem'].isin(temp['stem'])]
len(label_cross_df), len(clred_df), len(clred_df)/len(label_cross_df),

(6599176, 4855234, 0.7357333703480555)

In [176]:
thres = 0.15
temp = clred_df[(clred_df['bx3']<thres) & (clred_df['bx4']<(thres+0.1) )
             & (clred_df['label_pred']==cls)]
len(temp)

859

In [177]:
clred_df = clred_df[~clred_df['stem'].isin(temp['stem'])]
len(label_cross_df), len(clred_df), len(clred_df)/len(label_cross_df),

(6599176, 4851958, 0.7352369447337062)

In [181]:
thres = 0.05
temp = clred_df[(clred_df['bx3']<thres) | (clred_df['bx4']<thres) 
             & (clred_df['label_pred']==cls)]
len(temp)

0

In [182]:
clred_df = clred_df[~clred_df['stem'].isin(temp['stem'])]
len(label_cross_df), len(clred_df), len(clred_df)/len(label_cross_df),

(6599176, 4851958, 0.7352369447337062)

##### 长宽比

In [200]:
thres = 1.4
temp = clred_df[(clred_df['bx_h'].values/clred_df['bx_w'].values<thres)
             & (clred_df['label_pred']==cls)]
len(temp)

25676

In [201]:
clred_df = clred_df[~clred_df['stem'].isin(temp['stem'])]
len(label_cross_df), len(clred_df), len(clred_df)/len(label_cross_df),

(6599176, 4654432, 0.7053050259608169)

#### 鞋子的位置

In [202]:
model_label_map = {
    0: 10,  # 鞋子
    1: 8,  # 包包
    2: 4,  # 上装
    3: 5,  # 裤子
    4: 6,  # 裙子
    5: 7,  # 连体装
}

In [223]:
cls = 5

In [227]:
thres = 0.7
temp = clred_df[(clred_df['bx2']>thres) 
             & (clred_df['label_pred']==cls)]
len(temp)

10160

In [228]:
clred_df = clred_df[~clred_df['stem'].isin(temp['stem'])]
len(label_cross_df), len(clred_df), len(clred_df)/len(label_cross_df),

(6599176, 4450660, 0.674426625384745)

#### bbox偏离中心的

In [238]:
thres = 0.2
temp = clred_df[(((clred_df['bx1']<thres)& (clred_df['bx2']<thres)) |
               ((clred_df['bx1']>1-thres)& (clred_df['bx2']>1-thres)) |
               ((clred_df['bx1']>1-thres)& (clred_df['bx2']<thres)) |
               ((clred_df['bx1']<thres)& (clred_df['bx2']>1-thres)))]
len(temp)

24127

In [239]:
clred_df = clred_df[~clred_df['stem'].isin(temp['stem'])]
len(label_cross_df), len(clred_df), len(clred_df)/len(label_cross_df),

(6599176, 4276565, 0.6480453014133886)

In [240]:
clred_df.describe()

,label_pred,bx1,bx2,bx3,bx4,conf,id_crop,width,heigth,conf_show,bx_h,bx_w,h/w
count,4.276565e+06,4.276565e+06,4.276565e+06,4.276565e+06,4.276565e+06,4.276565e+06,4.276565e+06,4.276565e+06,4.276565e+06,4.276565e+06,4.276565e+06,4.276565e+06,4.276565e+06
mean,1.549420e+00,4.973157e-01,6.853039e-01,2.135318e-01,2.426166e-01,8.169581e-01,1.724285e+00,8.922618e+02,1.185037e+03,8.155439e+00,2.868372e+02,1.897854e+02,1.583868e+00
std,1.553438e+00,9.936780e-02,2.066608e-01,1.228110e-01,1.581768e-01,1.252243e-01,1.268442e+00,2.916066e+02,3.628137e+02,1.285817e+00,2.143494e+02,1.288673e+02,7.001135e-01
min,0.000000e+00,2.536230e-02,2.934780e-02,5.000000e-02,4.000000e-02,5.000000e-01,0.000000e+00,5.010000e+02,5.010000e+02,5.000000e+00,5.000000e+01,2.515000e+01,2.500004e-01
25%,0.000000e+00,4.442030e-01,5.073240e-01,1.101450e-01,1.053140e-01,7.354250e-01,1.000000e+00,6.900000e+02,9.200000e+02,7.000000e+00,1.190002e+02,9.299970e+01,1.052670e+00
50%,1.000000e+00,4.984360e-01,6.835940e-01,1.880000e-01,2.009380e-01,8.579490e-01,2.000000e+00,6.900000e+02,1.035000e+03,9.000000e+00,2.236635e+02,1.580006e+02,1.457367e+00
75%,3.000000e+00,5.506940e-01,8.984920e-01,2.929690e-01,3.523740e-01,9.187690e-01,3.000000e+00,1.279000e+03,1.640000e+03,9.000000e+00,3.951303e+02,2.490003e+02,2.001156e+00
max,5.000000e+00,9.740740e-01,9.800000e-01,1.000000e+00,1.000000e+00,9.866390e-01,6.000000e+00,8.256000e+03,8.192000e+03,1.000000e+01,5.192221e+03,3.150000e+03,4.000000e+00


### 数据拷贝

In [14]:
import sys
sys.path.append('../')
from utils.data.transfer import copy_data_mkdir, run_task_multithread

In [15]:
clred_df.head(1)

,label_pred,bx1,bx2,bx3,bx4,conf,id_crop,label_p,stem,oss_sp,width,heigth,label_eng,crops_sp,crops_tp
0,0,0.373611,0.942361,0.083333,0.115278,0.880866,0,/nas/chenyi/datasets_cls/gallery_detect/galler...,69313640-af52-465e-88c8-c0a0099ca5c7-699544,/nas/oss/gallery/69/31/69313640-af52-465e-88c8...,933,933,shoe,/nas/chenyi/datasets_cls/gallery_detect/galler...,/nas/chenyi/datasets_cls/gallery_detect/galler...


In [44]:
input_list = [list(item) for item in zip(clred_final_df['crops_sp'].values, clred_final_df['crops_tp'].values)]

In [61]:
%%time
import shutil
def copy_data_mkdir(plist, mode='move'):
    ps = Path(plist[0])
    pt = Path(plist[1])
    if not pt.parent.is_dir():
        pt.parent.mkdir(parents=True, exist_ok=True)
    if ps.is_file() and (not pt.is_file()):
        if mode=='move':
            Path.rename(ps, pt)
        else:
            shutil.copyfile(ps, pt)

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 5.48 µs


In [72]:
from tqdm import tqdm
for plist in tqdm(input_list):
    copy_data_mkdir(plist)
    # break

100%|████████████████| 4276565/4276565 [08:50<00:00, 8064.84it/s]


In [73]:
# %%time           
# run_task_multithread(input_list[::-1], copy_data_mkdir, 20)